In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ged4py import GedcomReader
import igraph as ig
import plotly.offline as py
import plotly.graph_objs as go

In [2]:
file_path = 'Family_Tree.ged'
parser = GedcomReader(file_path, encoding="utf-8")

In [3]:
indi_list = []
for i, indi in enumerate(parser.records0("INDI")):
    indi_dict = {}
    indi_dict['id'] = int(indi.xref_id.split('@')[1][1:])-1
    #indi_dict['name'] = indi.name.format()
    indi_dict['gender'] = indi.sub_tag_value("SEX")
    indi_dict['birth_date'] = indi.sub_tag_value("BIRT/DATE")
    indi_dict['birth_place'] = indi.sub_tag_value("BIRT/PLAC")
    indi_dict['death_date'] = indi.sub_tag_value("DEAT/DATE")
    indi_dict['death_place'] = indi.sub_tag_value("DEAT/PLAC")
    indi_list.append(indi_dict)
    
df_indi = pd.DataFrame(indi_list).set_index('id').sort_index().fillna('Unknown')
df_indi.to_csv('family_tree_network.csv', index=False)

In [4]:
family_list = []
for i, fam in enumerate(parser.records0("FAM")):
    husband, wife = fam.sub_tag("HUSB"), fam.sub_tag("WIFE")
    if husband and wife: 
        family_list.append((int(husband.xref_id.split('@')[1][1:])-1, int(wife.xref_id.split('@')[1][1:])-1))
        family_list.append((int(wife.xref_id.split('@')[1][1:])-1, int(husband.xref_id.split('@')[1][1:])-1))
    children = fam.sub_tags("CHIL")
    for child in children:
        if husband:
            family_list.append((int(husband.xref_id.split('@')[1][1:])-1, int(child.xref_id.split('@')[1][1:])-1))
        if wife:
            family_list.append((int(wife.xref_id.split('@')[1][1:])-1, int(child.xref_id.split('@')[1][1:])-1))

In [5]:
g = ig.Graph(directed=False)
g.add_vertices(df_indi.index.max()+1)
g.add_edges(family_list)
g.delete_vertices(list(set(range(df_indi.index.max()))-set(df_indi.index)))
ig.summary(g)

IGRAPH U--- 210 420 -- 


In [6]:
threeD = True
edge_list = g.get_edgelist()

if threeD:
    layout = g.layout('fr3d')
    Xn3d = [layout[k][0] for k in range(len(indi_list))]
    Yn3d = [layout[k][1] for k in range(len(indi_list))]
    Zn3d = [layout[k][2] for k in range(len(indi_list))]
    
    Xe3d = []
    Ye3d = []
    Ze3d = []
    for e in edge_list:
        Xe3d.extend([layout[e[0]][0], layout[e[1]][0], None])
        Ye3d.extend([layout[e[0]][1], layout[e[1]][1], None])
        Ze3d.extend([layout[e[0]][2], layout[e[1]][2], None])
    
    trace1 = go.Scatter3d(x=Xe3d, y=Ye3d, z=Ze3d, mode='lines', line=dict(color='rgb(0,0,0)', width=1), hoverinfo='none')
    trace2 = go.Scatter3d(x=Xn3d, y=Yn3d, z=Zn3d, mode='markers', name='people',
                          marker=dict(symbol='circle', size=6, colorscale='plasma', 
                                      color=np.sqrt(np.array(Xn3d)**2 + np.array(Yn3d)**2 + np.array(Zn3d)**2),
                                      line=dict(color='rgb(50,50,50)', width=0.5)),
                          text=df_indi.index.astype(str) + ' ' + df_indi['birth_date'].astype(str) + '-' + df_indi['death_date'].astype(str),
                          hoverinfo='text')
else:
    layout = g.layout('fr')
    Xn = [layout[k][0] for k in range(len(indi_list))]
    Yn = [layout[k][1] for k in range(len(indi_list))]
    
    Xe = []
    Ye = []
    for e in edge_list:
        Xe.extend([layout[e[0]][0], layout[e[1]][0], None])
        Ye.extend([layout[e[0]][1], layout[e[1]][1], None])
    
    trace1 = go.Scatter(x=Xe, y=Ye, mode='lines', line=dict(color='rgb(0,0,0)', width=1), hoverinfo='none')
    trace2 = go.Scatter(x=Xn, y=Yn, mode='markers', name='people',
                        marker=dict(symbol='circle', size=6,
                                    colorscale='plasma', color=np.sqrt(np.array(Xn)**2 + np.array(Yn)**2),
                                    line=dict(color='rgb(50,50,50)', width=0.5)),
                        text=df_indi.index.astype(str) + ' ' + df_indi['birth_date'].astype(str) + '-' + df_indi['death_date'].astype(str),
                        hoverinfo='text')

axis = dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, title='')
pylayout = go.Layout(title="Family Tree Network", width=1550, height=750, showlegend=False,
                     scene=dict(xaxis=dict(axis), yaxis=dict(axis), zaxis=dict(axis)),
                     margin=dict(t=30), hovermode='closest',
                     annotations=[dict(showarrow=False, text='', xref='paper', yref='paper',
                                       x=0, y=0.1, xanchor='left', yanchor='bottom', font=dict(size=14))])

In [7]:
data = [trace1, trace2]
fig = go.Figure(data=data, layout=pylayout)

py.plot(fig, filename='family_tree_network.html')

'family_tree_network.html'